# Custom TFIDF vs SkLearn TFIDF

<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

### Corpus

In [1]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### SkLearn Implementation

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [3]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [4]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [5]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [6]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [7]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### Our custom implementation

In [8]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np

In [9]:
class custom_tfidf ():
    def __init__(self, max_features = 0):
        self.vocab = {}
        self.idf_ = []
        self.vocab_tfidf = []
        self.max_features = max_features
    
    #define fit function to get vocab and idf values
    def fit(self,dataset):
        if isinstance(dataset, (list,)): #check if dataset passed as list else reject
            for row in dataset: # for each review in the dataset
                u_word = []
                for word in row.split(" "): # for each word in the review. 
                    if len(word) < 1: #do not consider short words
                        continue
                    elif word not in u_word: # store unique words of document
                        u_word.append(word)
                for wrd in u_word: # update the word count in document for idf
                    if wrd in self.vocab:
                        self.vocab[wrd] += 1
                    else:
                        self.vocab[wrd] = 1
            #get the idf values corrosponding to every word in vocab                         
            self.vocab = {i:1+np.log((len(dataset)+1)/(j+1)) for i,j in self.vocab.items()}
            #implementaton of max_features functionality
            if self.max_features != 0:
                self.vocab = dict(sorted(self.vocab.items(), key=lambda x:x[1], reverse = True))#sort by values
                #pick top n values
                self.vocab = {j[0]:j[1] for i,j in enumerate(self.vocab.items()) if i < self.max_features}    
            self.vocab = dict(sorted(self.vocab.items(), key=lambda x:x[0])) #sort
            self.idf_ = list(self.vocab.values())
        else:
            print("you need to pass list of sentances")
    
    def get_feature_names(self):
        return list(self.vocab.keys())
    
    # define transform to get final tfid sparse matrix
    def transform(self,dataset):
        if isinstance(dataset, (list,)):#check if dataset passed as list else reject
            self.vocab_tfidf = self.vocab
            rows = []
            columns = []
            values = []
            for idx,row in enumerate(dataset): # for each review in the dataset
                count = 0
                for word,idf in self.vocab.items(): # for each word in the vocab. 
                    rw = row.split(" ")
                    if word in list(rw): #calculate tfidf if word present in document
                        #compute tfidf values
                        values.append(self.vocab_tfidf[word]*(rw.count(word)/len(list(rw))))
                        rows.append(idx)
                        columns.append(count)
                    count +=1
            #mormalize and return sparse matrix
            return normalize(csr_matrix((values, (rows,columns)), shape=(len(dataset),len(self.vocab))), norm='l2')       
        else:
            print("you need to pass list of sentances")



In [10]:
vect = custom_tfidf()
vect.fit(corpus)
custom_op = vect.transform(corpus)

In [11]:
print(vect.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [12]:
print(vect.idf_)

[1.916290731874155, 1.2231435513142097, 1.5108256237659907, 1.0, 1.916290731874155, 1.916290731874155, 1.0, 1.916290731874155, 1.0]


In [13]:
custom_op.shape

(4, 9)

In [14]:
print(custom_op[0])

  (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149


In [15]:
print(custom_op[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


In [16]:
# Below is the code to load the cleaned_strings pickle file 
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


In [17]:
vect = custom_tfidf(max_features=50)
vect.fit(corpus)
custom_op = vect.transform(corpus)

In [18]:
custom_op.shape

(746, 50)

In [19]:
print(vect.get_feature_names())

['aimless', 'artiness', 'attempting', 'aye', 'baby', 'buffet', 'conception', 'constructed', 'content', 'distressed', 'doomed', 'dozen', 'drifting', 'effort', 'emptiness', 'existent', 'fill', 'fit', 'florida', 'gerardo', 'graphics', 'highest', 'insane', 'massive', 'messages', 'muppets', 'nearly', 'number', 'occurs', 'overdue', 'owls', 'person', 'post', 'practically', 'properly', 'pulls', 'punches', 'puzzle', 'require', 'rocks', 'science', 'screenplay', 'solving', 'structure', 'superlative', 'teacher', 'th', 'tightly', 'unlockable', 'vitally']


In [20]:
print(vect.idf_)

[6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872]


In [21]:
print(custom_op[0].toarray())

[[0.57735027 0.         0.         0.         0.         0.
  0.         0.         0.         0.57735027 0.         0.
  0.57735027 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]
